# Automated Trading Bot - Live Testing Notebook

This notebook demonstrates how to test the automated trading bot system with simulated live market data.
It allows you to:
- Configure and start trading bots
- Simulate market conditions with random data
- Monitor bot performance in real-time
- Test different trading strategies

## 1. Setup and Imports

In [ ]:
# Standard imports
import asyncio
import json
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from typing import Dict, Any, List

# Visualization1
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# API client
import httpx
import websockets

# Configure display
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 10)
plt.style.use('seaborn-v0_8-darkgrid')

print("Setup complete!")

## 2. Configuration

In [ ]:
# API Configuration
API_BASE_URL = "http://localhost:8080"
WS_URL = "ws://localhost:8080/ws"

# Test Configuration
TEST_CONFIG = {
    "symbols": ["NIFTY", "BANKNIFTY"],
    "test_duration_minutes": 60,  # How long to run the test
    "tick_interval_seconds": 5,   # How often to generate new data
    "initial_prices": {
        "NIFTY": 20000,
        "BANKNIFTY": 45000
    },
    "volatility": {
        "NIFTY": 0.0005,      # 0.05% per tick
        "BANKNIFTY": 0.0007   # 0.07% per tick
    },
    "iv_range": {
        "min": 12,
        "max": 35
    }
}

# Bot Configuration Override (for testing)
BOT_TEST_CONFIG = {
    "short_straddle": {
        "enabled": True,
        "capital": 200000,
        "entry": {
            "iv_rank_min": 70,  # Lower for testing
            "dte_min": 20,
            "dte_max": 45
        },
        "exit": {
            "profit_target_pct": 30,
            "stop_loss_multiplier": 1.5
        }
    }
}

print("Configuration loaded")

## 3. Market Data Simulator

In [ ]:
class MarketDataSimulator:
    """Simulates realistic market data for testing"""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.current_prices = config["initial_prices"].copy()
        self.price_history = {symbol: [] for symbol in config["symbols"]}
        self.iv_levels = {symbol: random.uniform(15, 25) for symbol in config["symbols"]}
        
    def generate_tick(self, symbol: str) -> Dict[str, Any]:
        """Generate a new market tick"""
        # Random walk for price
        volatility = self.config["volatility"][symbol]
        price_change = np.random.normal(0, volatility)
        
        # Add trend component (slight upward bias)
        trend = 0.00001
        price_change += trend
        
        # Update price
        self.current_prices[symbol] *= (1 + price_change)
        
        # Generate OHLC
        high = self.current_prices[symbol] * (1 + abs(np.random.normal(0, volatility/2)))
        low = self.current_prices[symbol] * (1 - abs(np.random.normal(0, volatility/2)))
        
        # Random walk for IV
        iv_change = np.random.normal(0, 0.5)
        self.iv_levels[symbol] = np.clip(
            self.iv_levels[symbol] + iv_change,
            self.config["iv_range"]["min"],
            self.config["iv_range"]["max"]
        )
        
        # Create tick data
        tick = {
            "symbol": symbol,
            "ltp": round(self.current_prices[symbol], 2),
            "open": round(self.current_prices[symbol] * 0.999, 2),
            "high": round(high, 2),
            "low": round(low, 2),
            "close": round(self.current_prices[symbol], 2),
            "volume": random.randint(10000, 100000),
            "oi": random.randint(100000, 1000000),
            "iv": round(self.iv_levels[symbol], 2),
            "timestamp": datetime.now().isoformat()
        }
        
        # Store history
        self.price_history[symbol].append({
            "time": datetime.now(),
            "price": tick["ltp"],
            "iv": tick["iv"]
        })
        
        return tick
    
    def generate_option_chain(self, symbol: str) -> Dict[str, Any]:
        """Generate mock option chain data"""
        spot_price = self.current_prices[symbol]
        strike_gap = 50 if symbol == "NIFTY" else 100
        
        # Generate strikes around ATM
        atm_strike = round(spot_price / strike_gap) * strike_gap
        strikes = [atm_strike + (i * strike_gap) for i in range(-5, 6)]
        
        chain = {}
        
        # Generate data for each strike
        for strike in strikes:
            # Simple Black-Scholes approximation for demo
            distance = abs(strike - spot_price) / spot_price
            
            # Call option
            call_price = max(spot_price - strike, 0) + \
                        (spot_price * 0.01 * np.exp(-distance * 10))
            
            # Put option
            put_price = max(strike - spot_price, 0) + \
                       (spot_price * 0.01 * np.exp(-distance * 10))
            
            chain[strike] = {
                "CE": {
                    "ltp": round(call_price, 2),
                    "iv": self.iv_levels[symbol] + np.random.uniform(-2, 2),
                    "oi": random.randint(10000, 100000),
                    "volume": random.randint(1000, 10000)
                },
                "PE": {
                    "ltp": round(put_price, 2),
                    "iv": self.iv_levels[symbol] + np.random.uniform(-2, 2),
                    "oi": random.randint(10000, 100000),
                    "volume": random.randint(1000, 10000)
                }
            }
        
        return {
            "symbol": symbol,
            "spot_price": spot_price,
            "expiry": (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d"),
            "strikes": chain,
            "atm_strike": atm_strike,
            "iv_rank": self._calculate_iv_rank(symbol)
        }
    
    def _calculate_iv_rank(self, symbol: str) -> float:
        """Calculate mock IV rank"""
        current_iv = self.iv_levels[symbol]
        iv_min = self.config["iv_range"]["min"]
        iv_max = self.config["iv_range"]["max"]
        
        iv_rank = ((current_iv - iv_min) / (iv_max - iv_min)) * 100
        return round(iv_rank, 2)

# Initialize simulator
simulator = MarketDataSimulator(TEST_CONFIG)
print("Market simulator initialized")

## 4. API Client

In [ ]:
class TradingBotClient:
    """Client for interacting with Trading Bot API"""
    
    def __init__(self, base_url: str):
        self.base_url = base_url
        self.client = httpx.AsyncClient(base_url=base_url)
    
    async def get_status(self) -> Dict[str, Any]:
        """Get system status"""
        response = await self.client.get("/api/status")
        response.raise_for_status()
        return response.json()
    
    async def get_bots(self) -> List[Dict[str, Any]]:
        """Get all bots"""
        response = await self.client.get("/api/bots")
        response.raise_for_status()
        return response.json()
    
    async def start_bot(self, bot_name: str) -> Dict[str, Any]:
        """Start a bot"""
        response = await self.client.post(f"/api/bots/{bot_name}/start")
        response.raise_for_status()
        return response.json()
    
    async def stop_bot(self, bot_name: str) -> Dict[str, Any]:
        """Stop a bot"""
        response = await self.client.post(f"/api/bots/{bot_name}/stop")
        response.raise_for_status()
        return response.json()
    
    async def get_positions(self) -> List[Dict[str, Any]]:
        """Get all positions"""
        response = await self.client.get("/api/positions")
        response.raise_for_status()
        return response.json()
    
    async def get_performance(self) -> Dict[str, Any]:
        """Get performance metrics"""
        response = await self.client.get("/api/performance")
        response.raise_for_status()
        return response.json()
    
    async def close(self):
        """Close client"""
        await self.client.aclose()

# Initialize client
client = TradingBotClient(API_BASE_URL)
print("API client initialized")

## 5. Real-time Dashboard

In [ ]:
class TradingDashboard:
    """Real-time trading dashboard"""
    
    def __init__(self):
        self.fig = None
        self.price_data = {symbol: [] for symbol in TEST_CONFIG["symbols"]}
        self.pnl_data = []
        self.positions = []
        self.signals = []
    
    def create_dashboard(self):
        """Create the dashboard layout"""
        self.fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Price Chart', 'P&L Chart', 'Positions', 'Bot Status'),
            specs=[[{"type": "scatter"}, {"type": "scatter"}],
                   [{"type": "table"}, {"type": "table"}]]
        )
        
        # Initialize empty traces
        for symbol in TEST_CONFIG["symbols"]:
            self.fig.add_trace(
                go.Scatter(x=[], y=[], name=symbol, mode='lines'),
                row=1, col=1
            )
        
        # P&L trace
        self.fig.add_trace(
            go.Scatter(x=[], y=[], name='P&L', mode='lines+markers'),
            row=1, col=2
        )
        
        # Position table
        self.fig.add_trace(
            go.Table(
                header=dict(values=['Symbol', 'Type', 'Qty', 'Entry', 'Current', 'P&L']),
                cells=dict(values=[[], [], [], [], [], []])
            ),
            row=2, col=1
        )
        
        # Bot status table
        self.fig.add_trace(
            go.Table(
                header=dict(values=['Bot', 'State', 'Capital', 'Positions', 'P&L']),
                cells=dict(values=[[], [], [], [], []])
            ),
            row=2, col=2
        )
        
        # Update layout
        self.fig.update_layout(height=800, showlegend=True)
        
    def update_price_data(self, symbol: str, price: float, timestamp: datetime):
        """Update price chart"""
        self.price_data[symbol].append({'time': timestamp, 'price': price})
        
        # Keep last 100 points
        if len(self.price_data[symbol]) > 100:
            self.price_data[symbol] = self.price_data[symbol][-100:]
    
    def update_dashboard(self, market_data: Dict, bot_status: Dict, positions: List):
        """Update all dashboard components"""
        # Update price traces
        for i, symbol in enumerate(TEST_CONFIG["symbols"]):
            if symbol in self.price_data and self.price_data[symbol]:
                times = [d['time'] for d in self.price_data[symbol]]
                prices = [d['price'] for d in self.price_data[symbol]]
                self.fig.data[i].x = times
                self.fig.data[i].y = prices
        
        # Update P&L
        if bot_status and 'stats' in bot_status:
            self.pnl_data.append({
                'time': datetime.now(),
                'pnl': bot_status['stats'].get('total_pnl', 0)
            })
            
            if self.pnl_data:
                times = [d['time'] for d in self.pnl_data[-100:]]
                pnls = [d['pnl'] for d in self.pnl_data[-100:]]
                self.fig.data[len(TEST_CONFIG["symbols"])].x = times
                self.fig.data[len(TEST_CONFIG["symbols"])].y = pnls
        
        # Update positions table
        if positions:
            pos_data = {
                'Symbol': [p['symbol'] for p in positions],
                'Type': [p['position_type'] for p in positions],
                'Qty': [p['quantity'] for p in positions],
                'Entry': [f"{p['entry_price']:.2f}" for p in positions],
                'Current': [f"{p.get('current_price', 0):.2f}" for p in positions],
                'P&L': [f"{p.get('pnl', 0):.2f}" for p in positions]
            }
            self.fig.data[-2].cells.values = list(pos_data.values())
        
        # Update bot status table
        if bot_status and 'bots' in bot_status:
            bot_data = {
                'Bot': [],
                'State': [],
                'Capital': [],
                'Positions': [],
                'P&L': []
            }
            
            for bot_name, bot_info in bot_status['bots'].items():
                bot_data['Bot'].append(bot_name)
                bot_data['State'].append(bot_info.get('state', 'unknown'))
                bot_data['Capital'].append(f"{bot_info.get('capital', {}).get('available', 0):.0f}")
                bot_data['Positions'].append(str(bot_info.get('positions', 0)))
                bot_data['P&L'].append(f"{bot_info.get('performance', {}).get('total_pnl', 0):.2f}")
            
            self.fig.data[-1].cells.values = list(bot_data.values())
        
        return self.fig

# Initialize dashboard
dashboard = TradingDashboard()
dashboard.create_dashboard()
print("Dashboard created")

## 6. Live Trading Simulation

In [ ]:
async def run_trading_simulation(duration_minutes: int = 5):
    """Run the live trading simulation"""
    
    print(f"Starting {duration_minutes} minute trading simulation...")
    print("Make sure the trading bot system is running on port 8080!")
    
    start_time = datetime.now()
    end_time = start_time + timedelta(minutes=duration_minutes)
    
    # Check if system is running
    try:
        status = await client.get_status()
        print(f"✓ System is running. Active bots: {status['stats']['active_bots']}")
    except Exception as e:
        print(f"✗ Cannot connect to trading bot system: {e}")
        print("Please start the system with: python main.py")
        return
    
    # Start the Short Straddle bot if not running
    bots = await client.get_bots()
    short_straddle_bot = next((b for b in bots if 'straddle' in b['name'].lower()), None)
    
    if short_straddle_bot and short_straddle_bot['state'] != 'running':
        print("Starting Short Straddle Bot...")
        await client.start_bot(short_straddle_bot['name'])
        await asyncio.sleep(2)  # Wait for bot to start
    
    # Display dashboard
    from IPython.display import display
    import ipywidgets as widgets
    
    output = widgets.Output()
    display(output)
    
    tick_count = 0
    
    while datetime.now() < end_time:
        tick_count += 1
        
        # Generate market data for each symbol
        for symbol in TEST_CONFIG["symbols"]:
            # Generate tick
            tick = simulator.generate_tick(symbol)
            
            # Update dashboard price data
            dashboard.update_price_data(symbol, tick['ltp'], datetime.now())
            
            # Every 10 ticks, generate high IV scenario for testing
            if tick_count % 10 == 0:
                # Spike IV to trigger signals
                simulator.iv_levels[symbol] = random.uniform(28, 32)
                
                # Generate option chain
                option_chain = simulator.generate_option_chain(symbol)
                
                print(f"\n{symbol} - Price: {tick['ltp']:.2f}, "
                      f"IV: {tick['iv']:.2f}, IV Rank: {option_chain['iv_rank']:.1f}%")
        
        # Get updated status
        try:
            status = await client.get_status()
            positions = await client.get_positions()
            
            # Update dashboard
            with output:
                clear_output(wait=True)
                fig = dashboard.update_dashboard(
                    market_data=simulator.current_prices,
                    bot_status=status,
                    positions=positions
                )
                fig.show()
                
                # Display summary
                print(f"\nSimulation Time: {(datetime.now() - start_time).seconds}s / {duration_minutes * 60}s")
                print(f"Total P&L: ₹{status['stats']['total_pnl']:.2f}")
                print(f"Active Positions: {len(positions)}")
                
                if positions:
                    print("\nOpen Positions:")
                    for pos in positions:
                        print(f"  {pos['symbol']} {pos['position_type']}: "
                              f"Qty={pos['quantity']}, Entry={pos['entry_price']:.2f}, "
                              f"P&L={pos.get('pnl', 0):.2f}")
        
        except Exception as e:
            print(f"Error updating status: {e}")
        
        # Wait for next tick
        await asyncio.sleep(TEST_CONFIG["tick_interval_seconds"])
    
    print("\n" + "="*50)
    print("SIMULATION COMPLETE")
    print("="*50)
    
    # Final summary
    final_status = await client.get_status()
    final_positions = await client.get_positions()
    
    print(f"\nFinal Results:")
    print(f"Total Trades: {final_status['stats']['total_trades']}")
    print(f"Total P&L: ₹{final_status['stats']['total_pnl']:.2f}")
    print(f"Open Positions: {len(final_positions)}")
    
    # Performance by bot
    print("\nBot Performance:")
    for bot_name, bot_info in final_status['bots'].items():
        perf = bot_info.get('performance', {})
        print(f"  {bot_name}:")
        print(f"    Trades: {perf.get('total_trades', 0)}")
        print(f"    Win Rate: {perf.get('win_rate', 0):.1f}%")
        print(f"    P&L: ₹{perf.get('total_pnl', 0):.2f}")

## 7. Run the Simulation

In [ ]:
# Run a 5-minute simulation
await run_trading_simulation(duration_minutes=5)

## 8. Analyze Results

In [ ]:
# Get detailed performance analysis
async def analyze_performance():
    """Analyze trading performance"""
    
    performance = await client.get_performance()
    positions = await client.get_positions()
    
    # Create performance visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Price history
    ax = axes[0, 0]
    for symbol, history in simulator.price_history.items():
        if history:
            times = [h['time'] for h in history]
            prices = [h['price'] for h in history]
            ax.plot(times, prices, label=symbol)
    ax.set_title('Price Movement')
    ax.set_xlabel('Time')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(True)
    
    # 2. IV history
    ax = axes[0, 1]
    for symbol, history in simulator.price_history.items():
        if history:
            times = [h['time'] for h in history]
            ivs = [h['iv'] for h in history]
            ax.plot(times, ivs, label=f"{symbol} IV")
    ax.set_title('Implied Volatility')
    ax.set_xlabel('Time')
    ax.set_ylabel('IV (%)')
    ax.legend()
    ax.grid(True)
    
    # 3. P&L over time
    ax = axes[1, 0]
    if dashboard.pnl_data:
        times = [d['time'] for d in dashboard.pnl_data]
        pnls = [d['pnl'] for d in dashboard.pnl_data]
        ax.plot(times, pnls, 'g-', linewidth=2)
        ax.fill_between(times, 0, pnls, alpha=0.3, color='green' if pnls[-1] > 0 else 'red')
    ax.set_title('P&L Over Time')
    ax.set_xlabel('Time')
    ax.set_ylabel('P&L (₹)')
    ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax.grid(True)
    
    # 4. Trade summary
    ax = axes[1, 1]
    if performance.get('bot_performances'):
        bot_names = list(performance['bot_performances'].keys())
        pnls = [perf.get('total_pnl', 0) for perf in performance['bot_performances'].values()]
        
        colors = ['green' if pnl > 0 else 'red' for pnl in pnls]
        ax.bar(bot_names, pnls, color=colors, alpha=0.7)
        ax.set_title('P&L by Bot')
        ax.set_xlabel('Bot')
        ax.set_ylabel('P&L (₹)')
        ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    print("\nDetailed Performance Analysis")
    print("=" * 50)
    
    print(f"\nOverall Performance:")
    print(f"  Total P&L: ₹{performance.get('total_pnl', 0):.2f}")
    print(f"  Total Trades: {performance.get('total_trades', 0)}")
    print(f"  Win Rate: {performance.get('win_rate', 0):.1f}%")
    print(f"  Max Drawdown: {performance.get('max_drawdown', 0):.1f}%")
    
    if positions:
        print(f"\nOpen Positions Analysis:")
        total_locked = sum(pos.get('entry_price', 0) * pos.get('quantity', 0) for pos in positions)
        total_unrealized = sum(pos.get('pnl', 0) for pos in positions)
        print(f"  Total Positions: {len(positions)}")
        print(f"  Capital Locked: ₹{total_locked:.2f}")
        print(f"  Unrealized P&L: ₹{total_unrealized:.2f}")

# Run the analysis
await analyze_performance()

## 9. Test Different Scenarios

In [ ]:
# Test high volatility scenario
async def test_high_volatility_scenario():
    """Test bot behavior in high volatility"""
    print("Testing HIGH VOLATILITY scenario...")
    
    # Increase volatility
    original_vol = TEST_CONFIG["volatility"].copy()
    TEST_CONFIG["volatility"]["NIFTY"] *= 3
    TEST_CONFIG["volatility"]["BANKNIFTY"] *= 3
    
    # Increase IV levels
    for symbol in TEST_CONFIG["symbols"]:
        simulator.iv_levels[symbol] = random.uniform(30, 35)
    
    # Run short simulation
    await run_trading_simulation(duration_minutes=2)
    
    # Restore original volatility
    TEST_CONFIG["volatility"] = original_vol

# Test market crash scenario
async def test_market_crash_scenario():
    """Test bot behavior during market crash"""
    print("Testing MARKET CRASH scenario...")
    
    # Simulate 5% drop
    for symbol in TEST_CONFIG["symbols"]:
        simulator.current_prices[symbol] *= 0.95
        simulator.iv_levels[symbol] = 35  # Spike IV
    
    await run_trading_simulation(duration_minutes=2)

# Run scenario tests
print("Choose a scenario to test:")
print("1. High Volatility")
print("2. Market Crash")
print("3. Normal Market")

# Uncomment to run specific scenario
# await test_high_volatility_scenario()
# await test_market_crash_scenario()

## 10. Cleanup

In [ ]:
# Stop all bots and cleanup
async def cleanup():
    """Stop all bots and cleanup resources"""
    try:
        # Get all bots
        bots = await client.get_bots()
        
        # Stop running bots
        for bot in bots:
            if bot['state'] == 'running':
                print(f"Stopping {bot['name']}...")
                await client.stop_bot(bot['name'])
        
        # Close client
        await client.close()
        
        print("\nCleanup complete!")
        
    except Exception as e:
        print(f"Error during cleanup: {e}")

# Run cleanup
await cleanup()

## Summary

This notebook demonstrates:

1. **Market Simulation**: Realistic price and volatility movements
2. **Bot Integration**: Starting and monitoring trading bots via API
3. **Real-time Dashboard**: Live visualization of prices, positions, and P&L
4. **Performance Analysis**: Detailed metrics and charts
5. **Scenario Testing**: High volatility and crash scenarios

### Key Features Tested:
- ✅ Automated signal generation based on IV rank
- ✅ Position management and tracking
- ✅ Real-time P&L calculation
- ✅ Risk management (position limits, stop loss)
- ✅ Multi-bot coordination

### Next Steps:
1. Add more sophisticated market scenarios
2. Implement backtesting with historical data
3. Add machine learning models for signal generation
4. Enhance risk management algorithms
5. Add more trading strategies